# Клиенты и счета

## Инициализация

In [19]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions as F
from src.main.etl import get_latest_rates

In [2]:
builder = SparkSession.builder.appName("project_ca") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "./jars/postgresql-42.5.0.jar,./jars/vertica-jdbc-9.0.0-0.jar")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/05/08 17:17:00 WARN Utils: Your hostname, prodesk resolves to a loopback address: 127.0.1.1; using 192.168.31.239 instead (on interface eno1)
23/05/08 17:17:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/maxim/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/maxim/.ivy2/cache
The jars for the packages stored in: /home/maxim/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-980b4012-6e3c-4214-b6b0-65d03eb62c4b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in local-m2-cache
:: resolution report :: resolve 341ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

23/05/08 17:17:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Загрузка данных

### Vertica

In [6]:
url = "jdbc:vertica://vertica:5433/"

properties = {"user": "dbadmin", "password": "", "database": "", "driver": "com.vertica.jdbc.Driver"}

df_rates = spark.read.format("JDBC").options(url=url, query="SELECT * FROM Rate", **properties).load()
df_rates.printSchema()
df_rates.show()

+--------+-----+----------+
|Currency| Rate|  RateDate|
+--------+-----+----------+
|     EU |91.27|2020-01-01|
|     EU |91.27|2020-01-02|
|     RUB|  1.0|2020-01-01|
|     RUB|  1.0|2020-01-02|
|     USD|80.23|2020-01-01|
|     USD|80.23|2020-01-02|
+--------+-----+----------+



In [9]:
df_clients = spark.read.format("JDBC").options(url=url, query="SELECT * FROM Clients", **properties).load()
df_clients.printSchema()
df_clients.show(10)

root
 |-- ClientId: long (nullable = true)
 |-- ClientName: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Form: string (nullable = true)
 |-- RegisterDate: date (nullable = true)

+--------+----------+----+------+------------+
|ClientId|ClientName|Type|  Form|RegisterDate|
+--------+----------+----+------+------------+
|       1|  Клиент 1|   Ф|-     |  2020-11-01|
|       2|  Клиент 2|   Ю|   ООО|  2020-11-02|
|       3|  Клиент 3|   Ф|-     |  2020-11-03|
|       4|  Клиент 4|   Ю|  ИП  |  2020-11-04|
|       5|  Клиент 5|   Ф|-     |  2020-11-01|
|       6|  Клиент 6|   Ю|  АО  |  2020-11-02|
|       7|  Клиент 7|   Ф|-     |  2020-11-03|
|       8|  Клиент 8|   Ю|   ПАО|  2020-11-01|
|       9|  Клиент 9|   Ф|-     |  2020-11-02|
|      10| Клиент 10|   Ю|   ООО|  2020-11-03|
+--------+----------+----+------+------------+
only showing top 10 rows



In [10]:
df_accounts = spark.read.format("JDBC").options(url=url, query="SELECT * FROM Account", **properties).load()
df_accounts.printSchema()
df_accounts.show(10)

root
 |-- AccountID: long (nullable = true)
 |-- AccountNum: string (nullable = true)
 |-- ClientId: long (nullable = true)
 |-- DateOpen: date (nullable = true)

+---------+--------------------+--------+----------+
|AccountID|          AccountNum|ClientId|  DateOpen|
+---------+--------------------+--------+----------+
|        1|40702810927050000337|       1|2020-11-01|
|        2|40802810300000009067|       2|2020-11-02|
|        3|40802810300000009708|       3|2020-11-03|
|        4|40802810800000030701|       4|2020-11-04|
|        5|40802810300000011071|       5|2020-11-01|
|        6|40802810100000063339|       6|2020-11-02|
|        7|40702810823620000031|       7|2020-11-03|
|        8|40802810409260005894|       8|2020-11-04|
|        9|40802810905030000004|       9|2020-11-01|
|       10|40802810400020001097|      10|2020-11-02|
+---------+--------------------+--------+----------+
only showing top 10 rows



In [11]:
df_operations = spark.read.format("JDBC").options(url=url, query="SELECT * FROM Operation", **properties).load()
df_operations.printSchema()
df_operations.show(10)

root
 |-- AccountDB: long (nullable = true)
 |-- AccountCR: long (nullable = true)
 |-- DateOp: date (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Comment: string (nullable = true)

+---------+---------+----------+------+--------+--------------------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|
+---------+---------+----------+------+--------+--------------------+
|     5050|    10048|2020-11-02|2505.0|     EU |сигар, табачн, та...|
|     5060|    10058|2020-11-04|2506.0|     RUB|гсм, бензин, керо...|
|     5070|    10068|2020-11-02|2507.0|     USD|компью, монитор, ...|
|     5080|    10078|2020-11-04|2508.0|     EU |ювелир,  юв., лом...|
|     5090|    10088|2020-11-02|2509.0|     RUB|хоз, бытхим, подг...|
|     5100|    10098|2020-11-04|2510.0|     USD|турусл, турист, з...|
|     5110|    10108|2020-11-02|2511.0|     EU |торгвыруч, оплату...|
|     5120|    10118|2020-11-04|2512.0|     RUB|а/м, ам, автомо

### CSV

In [12]:
schema = StructType() \
      .add("ClientId",IntegerType(),True) \
      .add("ClientName",StringType(),True) \
      .add("Type",StringType(),True) \
      .add("Form",StringType(),True) \
      .add("RegisterDate",DateType(),True) \
      
df_clients = (
    spark.read
        .options(delimiter=';')
        .option("header",True)
        .schema(schema)
        .csv("./data/Clients.csv")
)

df_clients.printSchema(), df_clients.show(10)

root
 |-- ClientId: integer (nullable = true)
 |-- ClientName: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Form: string (nullable = true)
 |-- RegisterDate: date (nullable = true)

+--------+----------+----+----+------------+
|ClientId|ClientName|Type|Form|RegisterDate|
+--------+----------+----+----+------------+
|       1|  Клиент 1|   Ф|   -|  2020-11-01|
|       2|  Клиент 2|   Ю| ООО|  2020-11-02|
|       3|  Клиент 3|   Ф|   -|  2020-11-03|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|
|       5|  Клиент 5|   Ф|   -|  2020-11-01|
|       6|  Клиент 6|   Ю|  АО|  2020-11-02|
|       7|  Клиент 7|   Ф|   -|  2020-11-03|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|
|       9|  Клиент 9|   Ф|   -|  2020-11-02|
|      10| Клиент 10|   Ю| ООО|  2020-11-03|
+--------+----------+----+----+------------+
only showing top 10 rows



(None, None)

In [13]:
schema = StructType() \
      .add("AccountID",IntegerType(),True) \
      .add("AccountNum",StringType(),True) \
      .add("ClientId",IntegerType(),True) \
      .add("DateOpen",DateType(),True) \
      
df_accounts = (
    spark.read
        .options(delimiter=';')
        .option("header",True)
        .schema(schema)
        .csv("./data/Account.csv")
)

df_accounts.printSchema(), df_accounts.show(10)

root
 |-- AccountID: integer (nullable = true)
 |-- AccountNum: string (nullable = true)
 |-- ClientId: integer (nullable = true)
 |-- DateOpen: date (nullable = true)

+---------+--------------------+--------+----------+
|AccountID|          AccountNum|ClientId|  DateOpen|
+---------+--------------------+--------+----------+
|        1|40702810927050000337|       1|2020-11-01|
|        2|40802810300000009067|       2|2020-11-02|
|        3|40802810300000009708|       3|2020-11-03|
|        4|40802810800000030701|       4|2020-11-04|
|        5|40802810300000011071|       5|2020-11-01|
|        6|40802810100000063339|       6|2020-11-02|
|        7|40702810823620000031|       7|2020-11-03|
|        8|40802810409260005894|       8|2020-11-04|
|        9|40802810905030000004|       9|2020-11-01|
|       10|40802810400020001097|      10|2020-11-02|
+---------+--------------------+--------+----------+
only showing top 10 rows



(None, None)

In [15]:
schema = StructType() \
    .add("Currency", StringType(), True) \
    .add("Rate", StringType(), True) \
    .add("RateDate", DateType(), True) \

df_rates = (
    spark.read
    .options(delimiter=';')
    .option("header", True)
    .schema(schema)
    .csv("./data/Rate.csv")
)

df_rates.printSchema(), df_rates.show()

root
 |-- Currency: string (nullable = true)
 |-- Rate: string (nullable = true)
 |-- RateDate: date (nullable = true)

+--------+-----+----------+
|Currency| Rate|  RateDate|
+--------+-----+----------+
|     USD|80,23|2020-01-01|
|      EU|91,27|2020-01-01|
|     RUB|    1|2020-01-01|
|     USD|80,23|2020-01-02|
|      EU|91,27|2020-01-02|
|     RUB|    1|2020-01-02|
+--------+-----+----------+



(None, None)

In [17]:
schema = StructType() \
    .add("AccountDB", IntegerType(), True) \
    .add("AccountCR", IntegerType(), True) \
    .add("DateOp", DateType(), True) \
    .add("Amount", StringType(), True) \
    .add("Currency", StringType(), True) \
    .add("Comment", StringType(), True) \


df_operations = (
    spark.read
    .options(delimiter=';')
    .option("header", True)
    .schema(schema)
    .csv("./data/Operation.csv")
)

df_operations.printSchema(), df_operations.show(10)

root
 |-- AccountDB: integer (nullable = true)
 |-- AccountCR: integer (nullable = true)
 |-- DateOp: date (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Comment: string (nullable = true)

+---------+---------+----------+------+--------+--------------------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|
+---------+---------+----------+------+--------+--------------------+
|        1|     4999|2020-11-01|   0,1|     RUB|а/м, а\м, автомоб...|
|        2|     5000|2020-11-02|   0,2|     USD| сою, соя, зерно,...|
|        3|     5001|2020-11-03|   0,3|      EU|сигар, табачн, та...|
|        4|     5002|2020-11-04|   0,4|     RUB|гсм, бензин, керо...|
|        5|     5003|2020-11-01|   0,5|     USD|компью, монитор, ...|
|        6|     5004|2020-11-02|   0,6|      EU|ювелир,  юв., лом...|
|        7|     5005|2020-11-03|   0,7|     RUB|хоз, бытхим, подг...|
|        8|     5006|2020-11-04|   0,8|     USD|турусл, т

(None, None)

In [23]:
# Загрузка списков из postgres

# url = "jdbc:postgresql://postgres:5432/postgres"
# creds = {"user": "postgres", "password": "myPassword", "driver": "org.postgresql.Driver"}
# lists = spark.read.jdbc(url, "de_sprint.lists", properties = creds).collect()

# list1 = lists[0][1]
# list2 = lists[1][1]

# print(list1, end='\n\n')
# print(list2)

In [24]:
# для теста

list1 = "%а/м%, %а\м%, %автомобиль %, %автомобили %, %транспорт%, %трансп%средс%, %легков%, %тягач%, %вин%, %vin%,%viн:%, %fоrd%, %форд%,%кiа%, %кия%, %киа%%мiтsuвisнi%, %мицубиси%, %нissан%, %ниссан%, %sсанiа%, %вмw%, %бмв%, %аudi%, %ауди%, %jеер%, %джип%, %vоlvо%, %вольво%, %тоyота%, %тойота%, %тоиота%, %нyuнdаi%, %хендай%, %rенаulт%, %рено%, %реugеот%, %пежо%, %lаdа%, %лада%, %dатsuн%, %додж%, %меrсеdеs%, %мерседес%, %vоlкswаgен%, %фольксваген%, %sкоdа%, %шкода%, %самосвал%, %rover%, %ровер%"
list2= "%сою%, %соя%, %зерно%, %кукуруз%, %масло%, %молок%, %молоч%, %мясн%, %мясо%, %овощ%, %подсолн%, %пшениц%, %рис%, %с/х%прод%, %с/х%товар%, %с\х%прод%, %с\х%товар%, %сахар%, %сельск%прод%, %сельск%товар%, %сельхоз%прод%, %сельхоз%товар%, %семен%, %семечк%, %сено%, %соев%, %фрукт%, %яиц%, %ячмен%, %картоф%, %томат%, %говя%, %свин%, %курин%, %куриц%, %рыб%, %алко%, %чаи%, %кофе%, %чипс%, %напит%, %бакале%, %конфет%, %колбас%, %морож%, %с/м%, %с\м%, %консерв%, %пищев%, %питан%, %сыр%, %макарон%, %лосос%, %треск%, %саир%, % филе%, % хек%, %хлеб%, %какао%, %кондитер%, %пиво%, %ликер%"

## Подготовка данных

In [20]:
df_rates = df_rates.withColumn("Rate", F.regexp_replace("Rate", ",", ".").cast(DoubleType()))

df_rates = get_latest_rates(df_rates)

df_rates.printSchema(), df_rates.show(10)

root
 |-- Currency: string (nullable = true)
 |-- RateDate: date (nullable = true)
 |-- Rate: double (nullable = true)

+--------+----------+-----+
|Currency|  RateDate| Rate|
+--------+----------+-----+
|      EU|2020-01-02|91.27|
|     RUB|2020-01-02|  1.0|
|     USD|2020-01-02|80.23|
+--------+----------+-----+



(None, None)

In [21]:
df_operations = df_operations.withColumn(
    "Amount", F.regexp_replace("Amount", ",", ".").cast(DoubleType()))

df_operations = df_operations.join(
    df_rates, df_operations.Currency == df_rates.Currency)

# всё в рублях
df_operations = (
    df_operations
    .withColumn("Amount", F.round(F.col("Amount") * F.col("Rate"), 2))
    .select("AccountDB", "AccountCR", "DateOp", "Amount", "Comment")
)

df_operations.printSchema(), df_operations.show(10)

root
 |-- AccountDB: integer (nullable = true)
 |-- AccountCR: integer (nullable = true)
 |-- DateOp: date (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Comment: string (nullable = true)

+---------+---------+----------+------+--------------------+
|AccountDB|AccountCR|    DateOp|Amount|             Comment|
+---------+---------+----------+------+--------------------+
|        1|     4999|2020-11-01|   0.1|а/м, а\м, автомоб...|
|        2|     5000|2020-11-02| 16.05| сою, соя, зерно,...|
|        3|     5001|2020-11-03| 27.38|сигар, табачн, та...|
|        4|     5002|2020-11-04|   0.4|гсм, бензин, керо...|
|        5|     5003|2020-11-01| 40.12|компью, монитор, ...|
|        6|     5004|2020-11-02| 54.76|ювелир,  юв., лом...|
|        7|     5005|2020-11-03|   0.7|хоз, бытхим, подг...|
|        8|     5006|2020-11-04| 64.18|турусл, турист, з...|
|        9|     5007|2020-11-01| 82.14|торгвыруч, оплату...|
|       10|     5008|2020-11-02|   1.0|а/м, а\м, автомоб...|
+---

(None, None)

In [25]:
# Будем делать только на первую дату

data_mart_date = "2020-11-01"

In [26]:
debet =  (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .groupBy("AccountDB")
    .agg(
        F.round(F.sum(F.col("Amount")), 2).alias("SumAmountDB")
    )
)

debet.show(5), debet.count()

+---------+-----------+
|AccountDB|SumAmountDB|
+---------+-----------+
|      833|  877595.71|
|     1645| 1283174.66|
|     1829|  902867.22|
|     3749|  951583.38|
|     3997| 1345448.57|
+---------+-----------+
only showing top 5 rows



(None, 5000)

In [27]:
credit =  (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .groupBy("AccountCR")
    .agg(
        F.round(F.sum(F.col("Amount")), 2).alias("SumAmountCR")
    )
)

credit.show(5), credit.count()

+---------+-----------+
|AccountCR|SumAmountCR|
+---------+-----------+
|     5803|  1260934.0|
|     9427| 1356886.63|
|    10623| 1547420.02|
|    12027|  1595717.6|
|    12799| 1446167.08|
+---------+-----------+
only showing top 5 rows



(None, 5000)

## Витрины

### 1. Витрина corporate_payments

In [28]:
# AccountId	        ИД счета
# ClientId	        Ид клиента счета
# PaymentAmt	    Сумма операций по счету, где счет клиента указан в дебете проводки
# EnrollementAmt	Сумма операций по счету, где счет клиента указан в кредите проводки
# TaxAmt		    Сумму операций, где счет клиента указан в дебете, и счет кредита 40702
# ClearAmt	        Сумма операций, где счет клиента указан в кредите, и счет дебета 40802
# CarsAmt		    Сумма операций, где счет клиента указан в дебете проводки и назначение платежа не содержит слов по маскам Списка 1
# FoodAmt		    Сумма операций, где счет клиента указан в кредите проводки и назначение платежа содержит слова по Маскам Списка 2
# FLAmt		        Сумма операций с физ. лицами. Счет клиента указан в дебете проводки, а клиент в кредите проводки – ФЛ.
# CutoffDt	        Дата операции;


# Client
#  |-- ClientId: integer (nullable = true)
#  |-- ClientName: string (nullable = true)
#  |-- Type: string (nullable = true)
#  |-- Form: string (nullable = true)
#  |-- RegisterDate: date (nullable = true)

# Account
#  |-- AccountID: integer (nullable = true)
#  |-- AccountNum: string (nullable = true)
#  |-- ClientId: integer (nullable = true)
#  |-- DateOpen: date (nullable = true)

# Rate
#  |-- Currency: string (nullable = true)
#  |-- RateDate: date (nullable = true)
#  |-- Rate: double (nullable = true)

# Operations
#  |-- AccountDB: integer (nullable = true)
#  |-- AccountCR: integer (nullable = true)
#  |-- DateOp: date (nullable = true)
#  |-- Amount: double (nullable = true)
#  |-- Currency: string (nullable = true)
#  |-- Comment: string (nullable = true)

from functools import reduce

grouped_payment = (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .join(df_accounts, df_accounts.AccountID == df_operations.AccountDB)
    .groupBy("AccountDB")
    .agg(
        F.sum("Amount").alias("PaymentAmt")
    )
)

grouped_tax = (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .join(df_accounts, (df_accounts.AccountID == df_operations.AccountCR) & (df_accounts.AccountNum.startswith("40702")))
    .groupBy("AccountDB")
    .agg(
        F.sum("Amount").alias("TaxAmt")
    )
)

grouped_cars = (
    df_operations
    .where((df_operations.DateOp == data_mart_date) & (~reduce(lambda a, b: a | b, (df_operations.Comment.like(pat.replace("\\", "\\\\")) for pat in list1.split(",")))))
    .groupBy("AccountDB")
    .agg(
        F.sum("Amount").alias("CarsAmt")
    )
)


grouped_enrollement = (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .groupBy("AccountCR")
    .agg(
        F.sum("Amount").alias("EnrollementAmt")
    )
)

grouped_clear = (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .join(df_accounts, (df_accounts.AccountID == df_operations.AccountDB) & (df_accounts.AccountNum.startswith("40802")))
    .groupBy("AccountCR")
    .agg(
        F.sum("Amount").alias("ClearAmt")
    )
)

grouped_food = (
    df_operations
    .where((df_operations.DateOp == data_mart_date) & (~reduce(lambda a, b: a | b, (df_operations.Comment.like(pat.replace("\\", "\\\\")) for pat in list2.split(",")))))
    .groupBy("AccountCR")
    .agg(
        F.sum("Amount").alias("FoodAmt")
    )
)


grouped_fl = (
    df_operations
    .where(df_operations.DateOp == data_mart_date)
    .join(df_accounts.alias("acc_db"), F.col("acc_db.AccountID") == df_operations.AccountDB)
    .join(df_accounts.alias("acc_cr"), F.col("acc_cr.AccountID") == df_operations.AccountCR)
    .join(df_clients.where(F.col("Type") == "Ф").alias("cl_cr"), F.col("acc_db.ClientId") == F.col("cl_cr.ClientId"))
    .groupBy("AccountDB")
    .agg(
        F.sum("Amount").alias("FLAmt")
    )
)


corporate_payments = (
    df_accounts
    .join(grouped_payment, df_accounts.AccountID == grouped_payment.AccountDB, "left")
    .join(grouped_tax, df_accounts.AccountID == grouped_tax.AccountDB, "left")
    .join(grouped_cars, df_accounts.AccountID == grouped_cars.AccountDB, "left")
    .join(grouped_enrollement, df_accounts.AccountID == grouped_enrollement.AccountCR, "left")
    .join(grouped_clear, df_accounts.AccountID == grouped_clear.AccountCR, "left")
    .join(grouped_food, df_accounts.AccountID == grouped_food.AccountCR, "left")
    .join(grouped_fl, df_accounts.AccountID == grouped_fl.AccountDB, "left")
    .select("AccountID", "ClientId", 
            F.when(F.col("PaymentAmt").isNotNull(), F.round(F.col("PaymentAmt"), 2)).otherwise(F.lit(0.0)).alias("PaymentAmt"), 
            F.when(F.col("EnrollementAmt").isNotNull(), F.round(F.col("EnrollementAmt"), 2)).otherwise(F.lit(0.0)).alias("EnrollementAmt"),
            F.when(F.col("TaxAmt").isNotNull(), F.round(F.col("TaxAmt"), 2)).otherwise(F.lit(0.0)).alias("TaxAmt"),
            F.when(F.col("ClearAmt").isNotNull(), F.round(F.col("ClearAmt"), 2)).otherwise(F.lit(0.0)).alias("ClearAmt"),
            F.when(F.col("CarsAmt").isNotNull(), F.round(F.col("CarsAmt"), 2)).otherwise(F.lit(0.0)).alias("CarsAmt"),
            F.when(F.col("FoodAmt").isNotNull(), F.round(F.col("FoodAmt"), 2)).otherwise(F.lit(0.0)).alias("FoodAmt"),
            F.when(F.col("FLAmt").isNotNull(), F.round(F.col("FLAmt"), 2)).otherwise(F.lit(0.0)).alias("FLAmt"))
)

# corporate_payments.orderBy("AccountID").show()

path = "data/" + data_mart_date + "/corporate_payments"

deltaTable = DeltaTable.forPath(spark, path)

deltaTable.history().show()

# deltaTable.restoreToVersion(0)

(
    deltaTable.alias("oldData")
    .merge(corporate_payments.alias("newData"), "oldData.AccountID = newData.AccountID")
    .whenMatchedUpdate(set={"AccountID": F.col("newData.AccountID"),
                            "ClientId": F.col("newData.ClientId"),
                            "PaymentAmt": F.col("newData.PaymentAmt"),
                            "EnrollementAmt": F.col("newData.EnrollementAmt"),
                            "TaxAmt": F.col("newData.TaxAmt"),
                            "ClearAmt": F.col("newData.ClearAmt"),
                            "CarsAmt": F.col("newData.CarsAmt"),
                            "FoodAmt": F.col("newData.FoodAmt"),
                            "FLAmt": F.col("newData.FLAmt")}
                       )
    .whenNotMatchedInsert(values={"AccountID": F.col("newData.AccountID"),
                                  "ClientId": F.col("newData.ClientId"),
                                  "PaymentAmt": F.col("newData.PaymentAmt"),
                                  "EnrollementAmt": F.col("newData.EnrollementAmt"),
                                  "TaxAmt": F.col("newData.TaxAmt"),
                                  "ClearAmt": F.col("newData.ClearAmt"),
                                  "CarsAmt": F.col("newData.CarsAmt"),
                                  "FoodAmt": F.col("newData.FoodAmt"),
                                  "FLAmt": F.col("newData.FLAmt")}
                          )
    .execute()
)

corporate_payments = deltaTable.toDF()

corporate_payments.orderBy("AccountID").show()

corporate_payments.write\
    .format("delta")\
    .option("path", path)\
    .mode("overwrite")\
    .save()


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      5|2023-04-09 17:59:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          4|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      4|2023-04-09 17:59:...|  null|    null|    MERGE|{predicate -> (ol...|null|    null|     null|          3|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      3|2

+---------+--------+----------+--------------+---------+----------+----------+----------+----------+
|AccountID|ClientId|PaymentAmt|EnrollementAmt|   TaxAmt|  ClearAmt|   CarsAmt|   FoodAmt|     FLAmt|
+---------+--------+----------+--------------+---------+----------+----------+----------+----------+
|        1|       1|1239646.48|           0.0|      0.0|       0.0|1239646.38|       0.0|1239646.48|
|        2|       2|       0.0|           0.0|      0.0|       0.0|       0.0|       0.0|       0.0|
|        3|       3|       0.0|           0.0|      0.0|       0.0|       0.0|       0.0|       0.0|
|        4|       4|       0.0|    1698092.02|      0.0|       0.0|       0.0|1698092.02|       0.0|
|        5|       5| 856586.88|           0.0|      0.0|       0.0| 375166.76|       0.0| 856586.88|
|        6|       6|       0.0|           0.0|      0.0|       0.0|       0.0|       0.0|       0.0|
|        7|       7|       0.0|           0.0|      0.0|       0.0|       0.0|       0.0|  

### 2. Витрина corporate_account

In [29]:
# AccountID	    ИД счета
# AccountNum	Номер счета
# DateOpen	    Дата открытия счета
# ClientId	    ИД клиента
# ClientName	Наименование клиента
# TotalAmt	    Общая сумма оборотов по счету. Считается как сумма PaymentAmt и EnrollementAmt
# CutoffDt	    Дата операции

total = debet.join(credit, debet.AccountDB == credit.AccountCR, "outer")
total = total.na.fill(value=0, subset=["SumAmountDB", "SumAmountCR"])
total = total.withColumn("AccountID", F.when(F.col("AccountDB").isNotNull(), F.col("AccountDB")).otherwise(F.col("AccountCR")))\
    .withColumn("TotalAmt", total.SumAmountDB + total.SumAmountCR)\
    .drop("AccountDB", "SumAmountDB", "AccountCR", "SumAmountCR")
total = total.join(df_accounts.alias("a"), total.AccountID == df_accounts.AccountID)\
    .select(total.AccountID, "ClientId", "TotalAmt")

# total.show()

sourceDF = (
    df_accounts
    .join(df_clients.alias("c"), df_accounts.ClientId == df_clients.ClientId)
    .join(total.alias("t"), total.AccountID == df_accounts.AccountID, "inner")
    .select(df_accounts.AccountID, "AccountNum", "DateOpen", "c.ClientId", "ClientName", "TotalAmt")    
)

# sourceDF.show()

path = "data/" + data_mart_date + "/corporate_account"

targetDF = DeltaTable.forPath(spark, path)

targetDF.history().show()

(
    targetDF.alias("oldData")
    .merge(sourceDF.alias("newData"), "oldData.AccountID = newData.AccountID")
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .whenNotMatchedBySourceDelete()
    .execute()
)

corporate_account = targetDF.toDF()

corporate_account.orderBy("AccountID").show()

corporate_account.write\
    .format("delta")\
    .option("path", path)\
    .mode("overwrite")\
    .save()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      6|2023-04-09 17:59:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          5|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      5|2023-04-09 17:59:...|  null|    null|    MERGE|{predicate -> (ol...|null|    null|     null|          4|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      4|2

### 3. Витрина corporate_info

In [30]:
# ClientId	    ИД клиента (PK)
# ClientName	Наименование клиента
# Type	        Тип клиента (ФЛ, ЮЛ)
# Form	        Организационно-правовая форма (ООО, ИП и т.п.)
# RegisterDate	Дата регистрации клиента
# TotalAmt	    Сумма операций по всем счетам клиент. Считается как сумма corporate_account.total_amt по всем счетам.
# CutoffDt	    Дата операции

grouped = total.groupby("ClientId").agg(F.sum("TotalAmt").alias("TotalAmt"))

sourceDF = (
    df_clients.join(grouped, df_clients.ClientId == grouped.ClientId)
    .select(df_clients.ClientId, "ClientName", "Type", "Form", "RegisterDate", F.round("TotalAmt", 2).alias("TotalAmt"))
)

# sourceDF.show()

path = "data/" + data_mart_date + "/corporate_info"

targetDF = DeltaTable.forPath(spark, path)

# targetDF.vacuum()

targetDF.history().show()

targetDF.detail().show()

(
    targetDF.alias("oldData")
    .merge(sourceDF.alias("newData"), "oldData.ClientId = newData.ClientId")
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .whenNotMatchedBySourceDelete()
    .execute()
)

corporate_info = targetDF.toDF()

corporate_info.orderBy("ClientId").show()

corporate_info.write\
    .format("delta")\
    .option("path", path)\
    .mode("overwrite")\
    .save()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      4|2023-04-09 18:02:...|  null|    null|  VACUUM END|{status -> COMPLE...|null|    null|     null|          3|SnapshotIsolation|         true|{numDeletedFiles ...|        null|Apache-Spark/3.3....|
|      3|2023-04-09 18:02:...|  null|    null|VACUUM START|{retentionCheckEn...|null|    null|     null|          2|SnapshotIsolation|         true|{numFilesToDelete...|        null|Ap

In [22]:
spark.stop()